In [ ]:
import s_index_processing as sip
import pandas as pd
import sys
import configparser
import plotly.express as px
from astropy.table import Table

# Read the configuration file
config = configparser.ConfigParser()
config.read('config.ini')

# Get the paths and settings from the configuration file
spectra_directory = config['PATHS']['spectra_directory']
ispec_dir = config['PATHS']['ispec_dir']
spectral_type = config['SETTINGS']['spectral_type']

sys.path.insert(0, ispec_dir)

# Process the files and get the results
results, errors = sip.process_spectra_files(spectra_directory, ispec_dir, spectral_type)

# Create a pandas DataFrame to display the results
df = pd.DataFrame(results)

# Display the DataFrame
print("Successfully processed files:")
display(df)

# Display any errors that occurred
if errors:
    print("\nErrors occurred while processing the following files:")
    error_df = pd.DataFrame(errors)
    display(error_df)

# Create an interactive plot and save reports
if not df.empty:
    fig = px.scatter(df, x='BJD', y='S_index', color='Name', 
                     title='S-index Time Series', 
                     hover_data=['Date', 'RV'])
    fig.show()

    # --- Generate and save reports ---
    # HTML Report
    with open('s_index_report.html', 'w') as f:
        f.write('<h1>S-index Analysis Report</h1>')
        f.write('<h2>Results</h2>')
        f.write(df.to_html())
        f.write('<h2>S-index Time Series Plot</h2>')
        f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
        if errors:
            f.write('<h2>Errors</h2>')
            f.write(pd.DataFrame(errors).to_html())
    print("\nHTML report saved as s_index_report.html")

    # LaTeX Table
    with open('s_index_table.tex', 'w') as f:
        f.write(df.to_latex(index=False))
    print("LaTeX table saved as s_index_table.tex")

    # FITS Table
    table = Table.from_pandas(df)
    table.write('s_index_table.fits', overwrite=True)
    print("FITS table saved as s_index_table.fits")